In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#df1
df1 = pd.read_csv('data/processed/human_raw_processed.csv')
df1.shape

(6010, 101)

In [3]:
df3 = pd.read_csv('data/processed/human_summary_processed.csv')
df3.shape

(4582, 70)

In [4]:
df1.columns

Index(['Source', 'DOI', 'Authors', 'Year', 'Species', 'Life Stage',
       'In Vitro or In Vivo?', 'Sex', 'Experiment Type', 'Exposure Route',
       ...
       'density_estimated_flag', 'density_unknown_flag',
       'particle_length_missing', 'sample_size_missing', 'sample_size_outlier',
       'dose_mg_per_l', 'dose_mg_per_l_missing', 'particles_per_ml',
       'particles_per_ml_missing', 'dose_info_available'],
      dtype='object', length=101)

In [5]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df1.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. Source
 2. DOI
 3. Authors
 4. Year
 5. Species
 6. Life Stage
 7. In Vitro or In Vivo?
 8. Sex
 9. Experiment Type
10. Exposure Route
11. Particle Mix?
12. Negative Control
13. Reference Particle
14. Exposure Media General
15. Exposure Media Specific
16. Exposure Media Additions
17. Solvent
18. Detergent
19. Temperature (Avg)
20. Exposure Duration (days)
21. Recovery (Days)
22. Number of Doses
23. Replicates
24. Sample Size
25. Dosing Frequency
26. particles/mL (master)
27. μg/mL (master)
28. Effect
29. Broad Endpoint Category
30. Specific Endpoint Category
31. Endpoint
32. Level of Biological Organization
33. Target Organelle, Cell, or Tissue
34. Polymer
35. Shape
36. Density (g/cm^3)
37. Density, reported or estimated
38. Charge
39. Zeta Potential (mV)
40. Zeta Potential Media
41. Functional Group
42. Particle Length (μm)
43. Size Category
44. Particle Volume (μm^3)
45. Particle Mass (mg)
46. Weathered or Biofouled?
47. Size Validated?
48. Polymer Val

In [6]:
print("LISTA COMPLETA DE VARIABLES")
for i, columna in enumerate(df3.columns):
    print(f"{i+1:2d}. {columna}")

LISTA COMPLETA DE VARIABLES
 1. DOI
 2. Year
 3. Species
 4. Life Stage
 5. In vitro/in vivo
 6. Sex
 7. Experiment Type
 8. Exposure Route
 9. Particle Mix?
10. Negative Control
11. Reference Particle
12. Exposure Media
13. Solvent
14. Detergent
15. Temperature (Avg)
16. Exposure Duration (days)
17. Number of Doses
18. Replicates
19. Sample Size
20. Dosing Frequency
21. Selected Dose
22. Effect
23. Broad Endpoint Category
24. Specific Endpoint Category
25. Endpoint
26. Level of Biological Organization
27. Target Organelle, Cell, or Tissue
28. Polymer
29. Shape
30. Density (g/cm^3)
31. Density, reported or estimated
32. Charge
33. Zeta Potential (mV)
34. Zeta Potential Media
35. Functional Group
36. Particle Length (μm)
37. Size Category
38. Particle Volume (μm^3)
39. Particle Mass (mg)
40. Weathered or Biofouled?
41. Size Validated?
42. Polymer Validated?
43. Shape Validated
44. Particle Source
45. Sodium Azide Present?
46. Screened for Chemical Contamination?
47. Particle Cleaning?
4

In [7]:
rename_dict_raw = {
    "In Vitro or In Vivo?": "In vitro/in vivo",

    # Unificamos todas las columnas de exposición
    "Exposure Media General": "Exposure Media (General)",
    "Exposure Media Specific": "Exposure Media (Specific)",
    "Exposure Media Additions": "Exposure Media (Additions)",
}

df1 = df1.rename(columns=rename_dict_raw)


# ===================================================
# 3. CREAR UNA ÚNICA COLUMNA "Exposure Media" EN RAW
# ===================================================

media_cols = [
    "Exposure Media (General)",
    "Exposure Media (Specific)",
    "Exposure Media (Additions)"
]

# Garantamos que los 3 existen (por seguridad)
for c in media_cols:
    if c not in df1.columns:
        df1[c] = np.nan

df1["Exposure Media"] = (
    df1["Exposure Media (General)"].fillna("") + ";" +
    df1["Exposure Media (Specific)"].fillna("") + ";" +
    df1["Exposure Media (Additions)"].fillna("")
)

# Limpieza estética: eliminar ";;" y ";" al inicio o final
df1["Exposure Media"] = (
    df1["Exposure Media"]
    .str.replace(";;", ";", regex=False)
    .str.replace("^;|;$", "", regex=True)
    .str.strip()
)

# Si queda vacío → not_reported
df1["Exposure Media"] = df1["Exposure Media"].replace("", "not_reported")


# ====================================================
# 4. FORZAR MISMO NOMBRE DE COLUMNAS EN SUMMARY
# ====================================================

# Algunas versiones del summary tienen "In Vitro/in vivo"
# Aseguramos que sea exactamente el mismo texto
df3 = df3.rename(columns={
    "In Vitro/in vivo": "In vitro/in vivo"
})


# ============================================
# 5. LISTA FINAL DE COLUMNAS A ALINEAR ENTRE RAW Y SUMMARY
# ============================================

cols_common = [
    "DOI", "Year", "Species", "Life Stage", "In vitro/in vivo", "Sex",
    "Experiment Type", "Exposure Route", "Particle Mix?", "Negative Control",
    "Reference Particle", "Exposure Media", "Solvent", "Detergent",
    "Temperature (Avg)", "Exposure Duration (days)", "Number of Doses",
    "Replicates", "Sample Size", "Dosing Frequency",
    "Effect", "Broad Endpoint Category", "Specific Endpoint Category",
    "Endpoint", "Level of Biological Organization",
    "Target Organelle, Cell, or Tissue",
    "Polymer", "Shape", "Density (g/cm^3)", "Density, reported or estimated",
    "Charge", "Zeta Potential (mV)", "Zeta Potential Media", "Functional Group",
    "Particle Length (μm)", "Size Category", "Particle Volume (μm^3)",
    "Particle Mass (mg)", "Weathered or Biofouled?", "Size Validated?",
    "Polymer Validated?", "Shape Validated", "Particle Source",
    "Sodium Azide Present?", "Screened for Chemical Contamination?",
    "Particle Cleaning?", "Solvent Rinse", "Background Contamination Monitored?",
    "Concentration Validated?", "Particle Behavior", "Uptake Validated?",
    "Organisms Fed?"
]

# Aseguramos que no haya espacios extra ni inconsistencias:
df1columns = df1.columns.str.strip()
df3.columns = df3.columns.str.strip()

# ======================================
# 6. REPORTAR COLUMNAS COINCIDENTES Y FALTANTES
# ======================================

common_in_raw = set(cols_common).intersection(df1.columns)
common_in_sum = set(cols_common).intersection(df3.columns)

missing_in_raw = set(cols_common) - set(df1.columns)
missing_in_sum = set(cols_common) - set(df3.columns)

print("\n=== COLUMNAS CORRECTAMENTE ALINEADAS ===")
print(sorted(list(common_in_raw.intersection(common_in_sum))))

print("\n=== FALTAN EN RAW ===")
print(sorted(list(missing_in_raw)))

print("\n=== FALTAN EN SUMMARY ===")
print(sorted(list(missing_in_sum)))

# =====================================================
# 7. EXPORTAR DATASETS HUMANOS LISTOS PARA MERGE
# =====================================================

df1.to_csv("human_raw_aligned.csv", index=False)
df3.to_csv("human_summary_aligned.csv", index=False)

print("\n✔ ARCHIVOS EXPORTADOS: human_raw_aligned.csv y human_summary_aligned.csv")



=== COLUMNAS CORRECTAMENTE ALINEADAS ===
['Background Contamination Monitored?', 'Broad Endpoint Category', 'Charge', 'Concentration Validated?', 'DOI', 'Density (g/cm^3)', 'Density, reported or estimated', 'Detergent', 'Dosing Frequency', 'Effect', 'Endpoint', 'Experiment Type', 'Exposure Duration (days)', 'Exposure Media', 'Exposure Route', 'Functional Group', 'In vitro/in vivo', 'Level of Biological Organization', 'Life Stage', 'Negative Control', 'Number of Doses', 'Organisms Fed?', 'Particle Behavior', 'Particle Cleaning?', 'Particle Length (μm)', 'Particle Mass (mg)', 'Particle Mix?', 'Particle Source', 'Particle Volume (μm^3)', 'Polymer', 'Polymer Validated?', 'Reference Particle', 'Replicates', 'Sample Size', 'Screened for Chemical Contamination?', 'Sex', 'Shape', 'Shape Validated', 'Size Category', 'Size Validated?', 'Sodium Azide Present?', 'Solvent', 'Solvent Rinse', 'Species', 'Specific Endpoint Category', 'Target Organelle, Cell, or Tissue', 'Temperature (Avg)', 'Uptake V

In [8]:
print("LISTA COMPLETA DE VARIABLES Y TIPOS")
for i, columna in enumerate(df3.columns):
    print(f"{i+1:2d}. {columna}  →  {df3[columna].dtype}")


LISTA COMPLETA DE VARIABLES Y TIPOS
 1. DOI  →  object
 2. Year  →  int64
 3. Species  →  object
 4. Life Stage  →  object
 5. In vitro/in vivo  →  object
 6. Sex  →  object
 7. Experiment Type  →  object
 8. Exposure Route  →  object
 9. Particle Mix?  →  object
10. Negative Control  →  object
11. Reference Particle  →  object
12. Exposure Media  →  object
13. Solvent  →  object
14. Detergent  →  object
15. Temperature (Avg)  →  float64
16. Exposure Duration (days)  →  float64
17. Number of Doses  →  int64
18. Replicates  →  float64
19. Sample Size  →  float64
20. Dosing Frequency  →  float64
21. Selected Dose  →  float64
22. Effect  →  object
23. Broad Endpoint Category  →  object
24. Specific Endpoint Category  →  object
25. Endpoint  →  object
26. Level of Biological Organization  →  object
27. Target Organelle, Cell, or Tissue  →  object
28. Polymer  →  object
29. Shape  →  object
30. Density (g/cm^3)  →  float64
31. Density, reported or estimated  →  object
32. Charge  →  object


In [9]:
df1.dtypes

Source                       object
DOI                          object
Authors                      object
Year                          int64
Species                      object
                             ...   
dose_mg_per_l_missing         int64
particles_per_ml            float64
particles_per_ml_missing      int64
dose_info_available           int64
Exposure Media               object
Length: 102, dtype: object

In [10]:
print("LISTA COMPLETA DE VARIABLES Y TIPOS")
for i, columna in enumerate(df1.columns):
    print(f"{i+1:2d}. {columna}  →  {df1[columna].dtype}")

LISTA COMPLETA DE VARIABLES Y TIPOS
 1. Source  →  object
 2. DOI  →  object
 3. Authors  →  object
 4. Year  →  int64
 5. Species  →  object
 6. Life Stage  →  object
 7. In vitro/in vivo  →  object
 8. Sex  →  object
 9. Experiment Type  →  object
10. Exposure Route  →  object
11. Particle Mix?  →  object
12. Negative Control  →  object
13. Reference Particle  →  object
14. Exposure Media (General)  →  object
15. Exposure Media (Specific)  →  object
16. Exposure Media (Additions)  →  object
17. Solvent  →  object
18. Detergent  →  object
19. Temperature (Avg)  →  float64
20. Exposure Duration (days)  →  float64
21. Recovery (Days)  →  float64
22. Number of Doses  →  int64
23. Replicates  →  float64
24. Sample Size  →  float64
25. Dosing Frequency  →  float64
26. particles/mL (master)  →  float64
27. μg/mL (master)  →  float64
28. Effect  →  object
29. Broad Endpoint Category  →  object
30. Specific Endpoint Category  →  object
31. Endpoint  →  object
32. Level of Biological Organizat

In [11]:
# Cargar datasets ya alineados en nombres
raw = pd.read_csv("human_raw_aligned.csv")
sumh = pd.read_csv("human_summary_aligned.csv")

# DTYPE maestro basado en tu dataset RAW (el más completo)
dtype_master = {
    "DOI": "object",
    "Year": "int64",
    "Species": "object",
    "Life Stage": "object",
    "In vitro/in vivo": "object",
    "Sex": "object",
    "Experiment Type": "object",
    "Exposure Route": "object",
    "Particle Mix?": "object",
    "Negative Control": "object",
    "Reference Particle": "object",
    "Exposure Media": "object",
    "Solvent": "object",
    "Detergent": "object",
    "Temperature (Avg)": "float64",
    "Exposure Duration (days)": "float64",
    "Number of Doses": "int64",
    "Replicates": "float64",
    "Sample Size": "float64",
    "Dosing Frequency": "float64",
    "Effect": "object",
    "Broad Endpoint Category": "object",
    "Specific Endpoint Category": "object",
    "Endpoint": "object",
    "Level of Biological Organization": "object",
    "Target Organelle, Cell, or Tissue": "object",
    "Polymer": "object",
    "Shape": "object",
    "Density (g/cm^3)": "float64",
    "Density, reported or estimated": "object",
    "Charge": "object",
    "Zeta Potential (mV)": "float64",
    "Zeta Potential Media": "object",
    "Functional Group": "object",
    "Particle Length (μm)": "float64",
    "Size Category": "object",
    "Particle Volume (μm^3)": "float64",
    "Particle Mass (mg)": "float64",
    "Weathered or Biofouled?": "object",
    "Size Validated?": "object",
    "Polymer Validated?": "object",
    "Shape Validated": "object",
    "Particle Source": "object",
    "Sodium Azide Present?": "object",
    "Screened for Chemical Contamination?": "object",
    "Particle Cleaning?": "object",
    "Solvent Rinse": "object",
    "Background Contamination Monitored?": "object",
    "Concentration Validated?": "object",
    "Particle Behavior": "object",
    "Uptake Validated?": "object",
    "Organisms Fed?": "object",

    # Flags
    "zeta_missing": "int64",
    "zeta_imputed_by_group": "int64",
    "zeta_imputed_by_global": "int64",
    "charge_missing": "int64",
    "in_vitro_flag": "bool",
    "particle_mass_missing": "int64",
    "particle_mass_imputed_by_group": "int64",
    "particle_mass_imputed_by_global": "int64",
    "particle_volume_missing": "int64",
    "density_reported_flag": "int64",
    "density_estimated_flag": "int64",
    "density_unknown_flag": "int64",
    "particle_length_missing": "int64",
    "sample_size_missing": "int64"
}

# Aplicar los dtypes donde ambas tablas comparten columna
shared_cols = set(raw.columns).intersection(sumh.columns)

for col in shared_cols:
    if col in dtype_master:
        target = dtype_master[col]
        try:
            raw[col] = raw[col].astype(target)
            sumh[col] = sumh[col].astype(target)
        except Exception as e:
            print(f"⚠ No se pudo convertir {col} a {target}: {e}")

print("\n✔ DTYPE alignment completed successfully.")

raw.to_csv("human_raw_aligned_types.csv", index=False)
sumh.to_csv("human_summary_aligned_types.csv", index=False)

print("✔ Exported updated files with harmonized dtypes.")



✔ DTYPE alignment completed successfully.
✔ Exported updated files with harmonized dtypes.


In [16]:
df11 = pd.read_csv('human_raw_aligned_types.csv')
df33 = pd.read_csv('human_summary_aligned_types.csv')

In [17]:
df11

,Source,DOI,Authors,Year,Species,Life Stage,In vitro/in vivo,Sex,Experiment Type,Exposure Route,...,density_unknown_flag,particle_length_missing,sample_size_missing,sample_size_outlier,dose_mg_per_l,dose_mg_per_l_missing,particles_per_ml,particles_per_ml_missing,dose_info_available,Exposure Media
0,search,10.1039/c9tx00147f,Amereh,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,...,0,0,0,0,NaN,1,NaN,1,0,water;distilled water;not_specified
1,search,10.1039/c9tx00147f,Amereh,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,...,0,0,0,0,NaN,1,NaN,1,0,water;distilled water;not_specified
2,search,10.1039/c9tx00147f,Amereh,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,...,0,0,0,0,NaN,1,NaN,1,0,water;distilled water;not_specified
3,search,10.1039/c9tx00147f,Amereh,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,...,0,0,0,0,NaN,1,NaN,1,0,water;distilled water;not_specified
4,search,10.1039/c9tx00147f,Amereh,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,...,0,0,0,0,NaN,1,NaN,1,0,water;distilled water;not_specified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6005,ToMEx 2.0,10.1016/j.ecoenv.2021.112837,Yang,2021,(Human) Homo sapiens,not_applicable,In Vitro,not_applicable,Particle Only,In Vitro,...,0,0,1,0,24.0,0,NaN,1,1,artificial_medium;RPMI 1610;not_specified
6006,ToMEx 2.0,10.1016/j.ecoenv.2021.112837,Yang,2021,(Human) Homo sapiens,not_applicable,In Vitro,not_applicable,Particle Only,In Vitro,...,0,0,1,0,48.0,0,NaN,1,1,artificial_medium;RPMI 1610;not_specified
6007,ToMEx 2.0,10.1016/j.ecoenv.2021.112837,Yang,2021,(Human) Homo sapiens,not_applicable,In Vitro,not_applicable,Particle Only,In Vitro,...,0,0,1,0,96.0,0,NaN,1,1,artificial_medium;RPMI 1610;not_specified
6008,ToMEx 2.0,10.1016/j.ecoenv.2021.112964,Kim,2021,(Human) Homo sapiens,not_applicable,In Vitro,not_applicable,Particle Only,In Vitro,...,0,0,0,0,NaN,1,NaN,1,0,none;none;not_specified


In [18]:
df33

,DOI,Year,Species,Life Stage,In vitro/in vivo,Sex,Experiment Type,Exposure Route,Particle Mix?,Negative Control,...,particle_mass_imputed_by_group,particle_mass_imputed_by_global,particle_volume_missing,volume_imputed_by_group,volume_imputed_by_global,density_reported_flag,density_estimated_flag,density_unknown_flag,particle_length_missing,sample_size_missing
0,10.1039/c9tx00147f,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
1,10.1039/c9tx00147f,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
2,10.1039/c9tx00147f,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
3,10.1039/c9tx00147f,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
4,10.1039/c9tx00147f,2019,(Rat) Rattus norvegicus,Adult,In Vivo,M,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577,10.1016/j.tox.2021.153059,2022,(Mouse) Mus musculus,Adult,In Vivo,M/F,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
4578,10.1016/j.tox.2021.153059,2022,(Mouse) Mus musculus,Adult,In Vivo,M/F,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
4579,10.1016/j.tox.2021.153059,2022,(Mouse) Mus musculus,Adult,In Vivo,M/F,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
4580,10.1016/j.tox.2021.153059,2022,(Mouse) Mus musculus,Adult,In Vivo,M/F,Particle Only,Gavage,no,yes,...,0,0,0,0,0,0,1,0,0,0
